### GKSL equation in Kossakowski form for the augmented Bloch vector 

In [1]:
include("../LiPoSID.jl")
using DynamicPolynomials
using LinearAlgebra

using QuantumOptics
basis = NLevelBasis(2)

using HDF5

using Dates

using Statistics

using TSSOS

According to the theorem proven by Kossakowski (Kossakowski, 1973) Markovian master equation could be rewritten for a two-level system $\rho \in M(2)$ as follows:

$
   \mathcal{K} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, 
$
where

$
\text{(i)} \quad \quad 
           f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k \\ \text{which means:} \notag \\ \quad
    \operatorname{Tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{Tr}(f_i) = 0 \notag 
$


and we can choose basis operators $f_i$ as halfs of Pauli matrices 
$\{ \sigma_x, \sigma_y, \sigma_z \}$:

$
    f_1 = \sigma_x/2  = 
    \begin{pmatrix}
        0 & 1 \\ 1 & 0
    \end{pmatrix} / 2 \\
    f_2 = \sigma_y/2  = \begin{pmatrix}
        0 & i \\ -i & 0
    \end{pmatrix} / 2\\
    f_3 = \sigma_z/2 = \begin{pmatrix}
        1 & 0 \\ 0 & -1
    \end{pmatrix} / 2
$

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2; fᴷ₂ = σʸ/2; fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.5im; 0.0 - 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

$
     \text{(ii)} \quad \quad \quad 
     H = \sum_1^3 h_i f_i, \quad h_i \in \mathbb{R}
$

which means we can use Hamiltonian of the form:

$
H = \begin{pmatrix} 
h_3   & h_1 + i h_2          \\
h_1 - i h_2 & -h_3
\end{pmatrix} / 2 \\
 = \begin{pmatrix} 
h_z   & h_x + i h_y          \\
h_x - i h_y & -h_z
\end{pmatrix} / 2 
$

In [3]:
@polyvar h[1:3]; h₁,h₂,h₃ = h[1],h[2],h[3]

Hˢʸᵐᵇ = [ h₃          h₁+im*h₂
          h₁-im*h₂   -h₃           ] / 2

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.5+0.0im)h₃                  (0.5+0.0im)h₁ + (0.0+0.5im)h₂
 (0.5+0.0im)h₁ + (0.0-0.5im)h₂  (-0.5+0.0im)h₃

$
     \text{(iii)} \quad \{C_{ij}\} = \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} 
=  \begin{pmatrix} 
\kappa_1   & - i a_3    &  i a_2 \\
i a_3       & \kappa_2   & -i a_1 \\
-i a_2      & i a_1      & \kappa_3 
\end{pmatrix}
$

$C =  \begin{pmatrix}
c_{11} & c_{12} & c_{13} \\
c_{21} & c_{22} & c_{23} \\
c_{31} & c_{32} & c_{33}
\end{pmatrix}$

In [4]:
@polyvar κ[1:3]; κ₁,κ₂,κ₃ = κ[1],κ[2],κ[3]
@polyvar a[1:3]; a₁,a₂,a₃ = a[1],a[2],a[3]

Cˢʸᵐᵇ = [  κ₁     -im*a₃     im*a₂
           im*a₃   κ₂       -im*a₁
          -im*a₂   im*a₁     κ₃     ] ;

$
    \text{(iv)} \quad \quad \quad \quad \gamma_1, \gamma_2, \gamma_3 \geq 0
$

If $\gamma = 0$ the evolution is just unitary, and in case $\gamma >0 $ <br>
in order for the evolution to be completely positive it is nessesary and sufficient that <br>
(denoting $\kappa_i = \gamma - 2 \gamma_i$):

$
    \kappa_1 + \kappa_2 + \kappa_3 \geq 0 \\
\kappa_1 \kappa_2  + \kappa_3 \kappa_1  + \kappa_1 \kappa_2  \geq a_1^2 + a_2^2 + a_3^2 \\
   \kappa_1 \kappa_2 \kappa_3 \geq  \kappa_1 a_1^2 + \kappa_2 a_2^2 + \kappa_3 a_3^2 
$

In [5]:
constr1 = κ₁ + κ₂ + κ₃ 
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - a₁^2 - a₂^2 - a₃^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a₁^2 - κ₂*a₂^2 - κ₃*a₃^2

constraints = [ h₃, κ₁, κ₂, κ₃, constr1, constr2, constr3] 

7-element Vector{Polynomial{true, Int64}}:
 h₃
 κ₁
 κ₂
 κ₃
 κ₁ + κ₂ + κ₃
 κ₁κ₂ + κ₁κ₃ + κ₂κ₃ - a₁² - a₂² - a₃²
 κ₁κ₂κ₃ - κ₁a₁² - κ₂a₂² - κ₃a₃²

Standard GKSL in Bloch vector form:

$
\frac{d\mathbf{r}}{dt} = \mathbf{\omega} \times \mathbf{r} - \Lambda \mathbf{r} + \mathbf{b}
$

where $\omega$ = $\{ h_1, h_2, h_3 \}$

$\mathbf{\omega} \times \mathbf{r} = \Omega \cdot \mathbf{r}  
= \begin{pmatrix}
0 & -h_z & h_y \\
h_z & 0 & -h_x \\
-h_y & h_x & 0
\end{pmatrix} \cdot \mathbf{r} $ 

Drift vector: $\mathbf{b} = 2 \sum_{i,j=1}^3 \operatorname{Im} C_{ij} \epsilon_{kij}$ with $\epsilon_{kij}$ - Levi-Civita symbol

$\mathbf{b} = 2 \operatorname{Im}  \begin{pmatrix} c_{23}-c_{32} \\ c_{31} - c_{13} \\ c_{12} - c_{21} \end{pmatrix} 
= 2 \operatorname{Im} \begin{pmatrix} -2i a_1 \\ -2i a_2 \\ -2i a_3 \end{pmatrix} = 2 \begin{pmatrix} -2 a_1 \\ -2 a_2 \\ -2 a_3 \end{pmatrix}$

Damping matrix:

$\Lambda_{kl} = 2 \sum_{i,j=1}^3 C_{ij} (\delta_{jk}\delta_{jl} - \frac{1}{2}\delta_{ij}\delta_{kl})$

$
\Lambda = 2 \begin{pmatrix}
c_{11} - \operatorname{Tr}(C) & c_{12} & c_{13} \\
c_{12} & c_{22} - \operatorname{Tr}(C) & c_{23} \\
c_{13} & c_{23} & c_{33} - \operatorname{Tr}(C)
\end{pmatrix}
$

$
\Lambda = 2 \left( C - \operatorname{Tr}(C) \, I \right)
$

In [6]:
Λ = 2*(Cˢʸᵐᵇ - tr(Cˢʸᵐᵇ)*I)

Ω = [  0   -h₃     h₂
       h₃   0     -h₁
      -h₂   h₁     0    ]  

b = -4 * [a₁ a₂ a₃]'

3×1 Matrix{Term{true, Int64}}:
 -4a₁
 -4a₂
 -4a₃

#### GKSL for augmented Bloch vector

$\frac{d\mathbf{\tilde{r}}}{dt} = \tilde{A_c} \mathbf{\tilde{r}}$


$\tilde{A_c} = 
\begin{pmatrix}
-\Lambda + \Omega & \mathbf{b} \\
\mathbf{0}^T &  0
\end{pmatrix}
$

In [7]:
Acˢʸᵐᵇ = [-Λ+Ω       b
          zeros(3)'  0]

4×4 Matrix{Polynomial{true, ComplexF64}}:
 (2.0+0.0im)κ₂ + (2.0+0.0im)κ₃                                        …  (-4.0+0.0im)a₁
 h₃ + (2.0+0.0im)κ₁ + (2.0+0.0im)κ₂ + (2.0+0.0im)κ₃ + (0.0-2.0im)a₃      (-4.0+0.0im)a₂
 -h₂ + (2.0+0.0im)κ₁ + (2.0+0.0im)κ₂ + (2.0+0.0im)κ₃ + (0.0+2.0im)a₂     (-4.0+0.0im)a₃
 0.0+0.0im                                                               0.0+0.0im

In [8]:
#using HDF5

#filename = "../MODELS/DMD_b4_LTI_trn4_2024-Sep-15_at_00-06.h5"

filename = "DMD-Bloch4D_SB_trn4_tst20_2024-Sep-16_at_14-30.h5"

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γᵢ = γ[3]

function read_LSID_Ac(γᵢ)

    h5open(filename, "r") do file
        # Navigate to the dataset within the group
        Ac = read(file[γᵢ*"/Ac"])
        
        # Now you can use the dataset as needed
        return(Ac)
    end
end 

Acˢⁱᵈ  = read_LSID_Ac(γᵢ)

4×4 Matrix{Float64}:
 -1.09155e-8   -25.1327        0.000161443   2.89442e-5
 25.1963        -0.795828     -0.000818289  -7.03521e-5
 -0.0366884     -0.00621641   -0.798674      0.79106
 -2.95082e-13   -4.98227e-13  -3.37554e-13  -2.0922e-14

Transfer function $G(s)$ should be the same for the all possible realisations ($A_c, C, \mathbf{x}(0)$) of the system.

Then transfer function for the some analytically assumed structure of matrices $\tilde{A_c}$ and $\tilde{C}$ could be equated to the transfer function calculated for the matrices  $\hat{A_c}^{(SID)}$ and $\hat{C}^{(SID)}$ estimated by ERA *system identification algorithm* algorithm:
$\begin{align}
   \tilde{C}(sI - \tilde{A_c})^{-1} \mathbf{x}(0) = \hat{C}^{(SID)}\left(sI - \hat{A_c}^{(SID)}\right)^{-1} \mathbf{x}(0)
\end{align}$ 

as we use augmented Bloch vectors as *internal state variable* then matrox $C$ is known 
$\tilde{C} = \hat{C}^{(SID)}$

$
\begin{pmatrix} \rho_{00} \\
\rho_{01} \\
\rho_{10} \\
\rho_{11} 
\end{pmatrix} 
= \frac{1}{2}
\begin{pmatrix} 
0 & 0 & 1 & 1 \\
1 & -i & 0 & 0 \\
1 & i & 0 & 0 \\
0 & 0 & -1 & 1 \\
\end{pmatrix}
\begin{pmatrix} 
\mathbf{b} \\
1
\end{pmatrix} $

Thus we can remole $C$ from consideration, and as we exactly know the initial states $\mathbf{x}(0)$ we can also remove it

$\begin{align}
   (sI - \tilde{A_c})^{-1} = \left(sI - \hat{A_c}^{(SID)}\right)^{-1}
\end{align}$ 

If matrices $sI - \tilde{A_c}$ and $sI - \hat{A_c}^{(SID)}$ are actually invertable, then:

$\begin{align}
   sI - \tilde{A_c} = sI - \hat{A_c}^{(SID)}
\end{align}$ 

Which means matrices  $\tilde{A_c}$ and $\hat{A_c}^{(SID)}$ are actually elementwise equal:

$\begin{align}
   \tilde{A_c} = \hat{A_c}^{(SID)}
\end{align}$ 

$\begin{align}
    \underset{ h_i, a_i, \kappa_i}{\text{minimize}} & \sum_{j=3}^N 
    \Bigg\|\tilde{A_c}^{(symb)} - \hat{A_c}^{(SID)}\Bigg\|^2_F, \notag\\
    \text{subject to } \\
    \kappa_1 + \kappa_2 + \kappa_3 \geq 0 \\
    \kappa_1 \kappa_2  + \kappa_3 \kappa_1  + \kappa_1 \kappa_2  \geq a_1^2 + a_2^2 + a_3^2 \\
   \kappa_1 \kappa_2 \kappa_3 \geq  \kappa_1 a_1^2 + \kappa_2 a_2^2 + \kappa_3 a_3^2 \\
\end{align}$


In [9]:
Acˢⁱᵈ[abs.(Acˢⁱᵈ) .< 1e-6] .= 0.0
D = Acˢⁱᵈ[1:3,1:4] - Acˢʸᵐᵇ[1:3,1:4]

obj = LiPoSID.frobenius_norm2(D)

(2.0 + 0.0im)h₁² + (2.0 + 0.0im)h₂² + (2.0 + 0.0im)h₃² + (32.0 + 0.0im)κ₁² + (56.0 + 0.0im)κ₁κ₂ + (56.0 + 0.0im)κ₁κ₃ + (32.0 + 0.0im)κ₂² + (56.0 + 0.0im)κ₂κ₃ + (32.0 + 0.0im)κ₃² + (24.0 + 0.0im)a₁² + (24.0 + 0.0im)a₂² + (24.0 + 0.0im)a₃² + (0.01079624353573517 + 0.0im)h₁ + (-0.07369977707941755 + 0.0im)h₂ + (-100.6581463952266 + 0.0im)h₃ + (6.297887272727243 + 0.0im)κ₁ + (3.1145769487947828 + 0.0im)κ₂ + (3.103191562085228 + 0.0im)κ₃ + (0.00023155339490666895 + 0.0im)a₁ + (-0.0005628165274150426 + 0.0im)a₂ + (6.328483672133771 + 0.0im)a₃ + (1268.4082100163819 + 0.0im)

In [10]:
obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))

2.0h₁² + 2.0h₂² + 2.0h₃² + 32.0κ₁² + 56.0κ₁κ₂ + 56.0κ₁κ₃ + 32.0κ₂² + 56.0κ₂κ₃ + 32.0κ₃² + 24.0a₁² + 24.0a₂² + 24.0a₃² + 0.01079624353573517h₁ - 0.07369977707941755h₂ - 100.6581463952266h₃ + 6.297887272727243κ₁ + 3.1145769487947828κ₂ + 3.103191562085228κ₃ + 0.00023155339490666895a₁ - 0.0005628165274150426a₂ + 6.328483672133771a₃ + 1268.4082100163819

In [11]:
#using TSSOS

function min_cs_tssos(p, constrs)

    coeffs = coefficients(p)
    reg_coef = 0

    pop =[p+reg_coef*sum(variables(p).^2), constrs...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    solution = variables(p) => ref_sol

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end

min_cs_tssos (generic function with 1 method)

In [12]:
sol, st = min_cs_tssos(obj, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[6, 1]
[1, 3]
-----------------------------------------------------------------------------
optimum = 0.0014273698316726445
Global optimality certified with relative optimality gap 0.005548%!

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.005276%!


(PolyVar{true}[h₁, h₂, h₃, κ₁, κ₂, κ₃, a₁, a₂, a₃] => [-0.0026990608839337926, 0.018424944269854387, 25.164536630383886, -1.1757087243570525e-5, -1.8157302857351571e-7, 3.622314790040512e-7, -1.302920830990817e-7, 3.166895374159483e-7, -0.00356095542462118], "GLOBAL")

In [13]:
Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
Cˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 -1.17571e-5+0.0im                 0.0+0.00356096im
         0.0-0.00356096im  -1.81573e-7+0.0im

In [14]:
Cˢⁱᵈ[2:3,2:3]

2×2 Matrix{ComplexF64}:
 -1.81573e-7+0.0im                0.0+1.30292e-7im
         0.0-1.30292e-7im  3.62231e-7+0.0im

In [15]:
Hˢⁱᵈ = convert.(ComplexF64,subs(Hˢʸᵐᵇ, sol))

2×2 Matrix{ComplexF64}:
     12.5823+0.0im         -0.00134953+0.00921247im
 -0.00134953-0.00921247im     -12.5823+0.0im

In [16]:
evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];
basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

In [17]:
println(" Zhang & Sarovar KOSSAKOWSKI form IDENTIFICATION w CONSTRAINED TSSOS from DMD Bloch4 Linear SID w objective QO simulation")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

relative_threshold = 1e-15

rltrs = string(convert(Int, floor(log10(relative_threshold))))

tests_data_file_name = "ZHANG_SAROVAR_DMD_B4_KOSSAK_CONSTR_TSSOS_treshold_1e"*rltrs*"_FROB_QO_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []
CoefRanges = []

for γᵢ in γ

    println("γ =  "*γᵢ)

    Acˢⁱᵈ  = read_LSID_Ac(γᵢ)
    
    elapsed_time = @timed begin

        Acˢⁱᵈ[abs.(Acˢⁱᵈ) .< 1e-6] .= 0.0
        obj = LiPoSID.frobenius_norm2(Acˢⁱᵈ[1:3,1:4] - Acˢʸᵐᵇ[1:3,1:4])
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
        sol, status = min_cs_tssos(obj, constraints)

    end

    @show minimum(abs.(coefficients(obj)))
    @show maximum(abs.(coefficients(obj)))
    push!(CoefRanges, LiPoSID.coefficient_range(obj))

    print(" status:", status)
    print(" runtime :", elapsed_time.time)

    Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))
    Cˢⁱᵈ = convert.(ComplexF64,subs(Cˢʸᵐᵇ, sol))
    epsilon = subs(h₃, sol) #ϵ

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)
        γ_group["status"] = status
        γ_group["runtime"] = elapsed_time.time

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = LiPoSID.read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        ρₒ = DenseOperator(basis, ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        effective_Lindblad = LiPoSID.get_lindblad_operators(convert.(ComplexF64, Cˢⁱᵈ), fᴷᴼᴺᴮ)
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(convert.(Float64, tᵗˢᵗ), ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #bˢⁱᵈ = LiPoSID.bloch(ρˢⁱᵈ)
        
        # Calculating fidelity
        
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)


        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
            #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            #init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = LiPoSID.TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = mean(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
    fid["CoefsRanges"] = convert.(Float64,CoefRanges)

end


println(tests_data_file_name)

 Zhang & Sarovar KOSSAKOWSKI form IDENTIFICATION w CONSTRAINED TSSOS from DMD Bloch4 Linear SID w objective QO simulation
γ =  0.079477
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[6, 1]
[1, 3]
-----------------------------------------------------------------------------
optimum = 1.4496927173337396e-5
Global optimality certified with relative optimality gap 0.000075%!
Global optimality certified with relative optimality gap 0.000102%!
minimum(abs.(coefficients(obj))) = 1.971979142062289e-5
maximum(abs.(coefficients(obj))) = 1263.5870815375833
 status:GLOBAL runtime :0.050505876
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9417993284930164
γ =  0.25133
*********************************** TSSOS ***********************************
TSSOS is launching...
---------------------------------------------------------